<h1 style="text-align:center">Telecom Customer Churn</h1>

## Part 2: Analysis of Customer Lifetime Value
Lifetime Value (LTV) is the total worth to a business of a customer over the period of their relationship to the company.

In [1]:
# importing required libraries
import sqlite3
import pandas as pd

In [2]:
# creating a sql connection
conn = sqlite3.connect("customer_churn.db")

In [3]:
# creating a dataframe from the customer_churn database
df = pd.read_sql("select * from churn_all;", conn)

In [4]:
# displaying the top 5 rows of the dataframe
df.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed0check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic0check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic0check,Yes,99.65,820.50,Yes
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,Yes,...,Yes,Yes,Yes,28,Month-to-month,Electronic0check,Yes,104.80,3046.05,Yes
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,Yes,...,No,Yes,Yes,49,Month-to-month,Bank0transfer0(automatic),Yes,103.70,5036.30,Yes


In [5]:
# displaying all the columns
print(df.columns)

Index(['CustomerID', 'Gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'State', 'Latitude', 'Longitude', 'ZipCode', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Tenure', 'Contract', 'PaymentMethod', 'PaperlessBilling',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')


---
### 1. Understanding the Customer Lifetime Value

#### Question 1: What was the average LTV of the customers who unsubscribed the service? And how long do customers usually stay in the service?

In [6]:
# extracting those customers who churned (churn = yes)
churn_df = pd.read_sql("select * from churn_all where Churn = 'Yes'", conn)

In [7]:
# displaying the top 5 rows
churn_df.head()

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed0check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic0check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic0check,Yes,99.65,820.50,Yes
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,Yes,...,Yes,Yes,Yes,28,Month-to-month,Electronic0check,Yes,104.80,3046.05,Yes
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,Yes,...,No,Yes,Yes,49,Month-to-month,Bank0transfer0(automatic),Yes,103.70,5036.30,Yes


In [8]:
# examining the distrubution of TotalCharges
churn_df["TotalCharges"].describe()

count    1869.000000
mean     1531.796094
std      1890.822994
min        18.850000
25%       134.500000
50%       703.550000
75%      2331.300000
max      8684.800000
Name: TotalCharges, dtype: float64

Around 20% of the TotalCharges are very high. So, we can divide the dataset to study each distribution.

In [8]:
# finding the 80th percentile of the data in TotalCharges
churn_df.TotalCharges.quantile(0.8)

2840.4100000000003

The Lifetime Value of 80% of the people who left the company was below $2840

In [9]:
# selecting all the customers who have TotalCharges less than or equal to $2840
pd.read_sql("select * from churn_all where TotalCharges <= 2840.41", conn)

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed0check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic0check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic0check,Yes,99.65,820.50,Yes
3,4190-MFLUW,Female,No,Yes,No,California,34.066367,-118.309868,90020,Yes,...,Yes,No,No,10,Month-to-month,Credit0card0(automatic),No,55.20,528.35,Yes
4,8779-QRDMV,Male,Yes,No,No,California,34.023810,-118.156582,90022,No,...,No,No,Yes,1,Month-to-month,Electronic0check,Yes,39.65,39.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,8456-QDAVC,Male,No,No,No,California,32.852947,-114.850784,92283,Yes,...,No,Yes,No,19,Month-to-month,Bank0transfer0(automatic),Yes,78.70,1495.10,No
4746,7750-EYXWZ,Female,No,No,No,California,34.159534,-116.425984,92284,No,...,Yes,Yes,Yes,12,One0year,Electronic0check,No,60.65,743.30,No
4747,2569-WGERO,Female,No,No,No,California,34.341737,-116.539416,92285,Yes,...,No internet service,No internet service,No internet service,72,Two0year,Bank0transfer0(automatic),Yes,21.15,1419.40,No
4748,6840-RESVB,Male,No,Yes,Yes,California,34.667815,-117.536183,92301,Yes,...,Yes,Yes,Yes,24,One0year,Mailed0check,Yes,84.80,1990.50,No


In [10]:
churn_df.query("TotalCharges <= 2840.41")

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,TechSupport,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,No,2,Month-to-month,Mailed0check,Yes,53.85,108.15,Yes
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,No,2,Month-to-month,Electronic0check,Yes,70.70,151.65,Yes
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,No,Yes,Yes,8,Month-to-month,Electronic0check,Yes,99.65,820.50,Yes
5,4190-MFLUW,Female,No,Yes,No,California,34.066367,-118.309868,90020,Yes,...,Yes,No,No,10,Month-to-month,Credit0card0(automatic),No,55.20,528.35,Yes
6,8779-QRDMV,Male,Yes,No,No,California,34.023810,-118.156582,90022,No,...,No,No,Yes,1,Month-to-month,Electronic0check,Yes,39.65,39.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863,1122-JWTJW,Male,No,Yes,Yes,California,32.698964,-115.886656,92259,Yes,...,No,No,No,1,Month-to-month,Mailed0check,Yes,70.65,70.65,Yes
1864,1699-HPSBG,Male,No,No,No,California,33.745746,-116.514215,92264,Yes,...,Yes,Yes,No,12,One0year,Electronic0check,Yes,59.80,727.80,Yes
1865,8775-CEBBJ,Female,No,No,No,California,32.790282,-115.689559,92273,Yes,...,No,No,No,9,Month-to-month,Bank0transfer0(automatic),Yes,44.20,403.35,Yes
1866,6894-LFHLY,Male,Yes,No,No,California,34.264124,-114.717964,92280,Yes,...,No,No,No,1,Month-to-month,Electronic0check,Yes,75.75,75.75,Yes


In [11]:
# dividing the data by the 80th percentile of the data from the TotalCharges variable
total_charges_under80 = churn_df.query("TotalCharges <= 2840.41")
total_charges_above80 = churn_df.query("TotalCharges > 2840.41")

In [12]:
# displaying the distribution of TotalCharges paid by people under 80th percentile
total_charges_under80.TotalCharges.describe()

count    1495.000000
mean      711.265819
std       766.848197
min        18.850000
25%        85.025000
50%       371.650000
75%      1128.225000
max      2839.650000
Name: TotalCharges, dtype: float64

In [13]:
# displaying the distribution of TotalCharges paid by people above 80th percentile
total_charges_above80.TotalCharges.describe()

count     374.000000
mean     4811.723262
std      1436.724288
min      2841.550000
25%      3522.462500
50%      4571.100000
75%      5891.212500
max      8684.800000
Name: TotalCharges, dtype: float64

In [14]:
# displaying the distribution of Tenure of people under 80th percentile
total_charges_under80.Tenure.describe()

count    1495.000000
mean        9.935117
std        10.742349
min         1.000000
25%         1.000000
50%         6.000000
75%        15.000000
max        61.000000
Name: Tenure, dtype: float64

In [15]:
# displaying the distribution of Tenure of people above 80th percentile
total_charges_above80.Tenure.describe()

count    374.000000
mean      50.133690
std       12.334841
min       27.000000
25%       40.000000
50%       49.500000
75%       60.000000
max       72.000000
Name: Tenure, dtype: float64

Answer:<br>
<i>The average LTV of 80% of the customers who unsubscribed is ~711 dollars and their tenure is ~10 months. On the other hand, the average LTV of top 20% of the customers who unsubscribed is ~4811 dollars and their tenure is ~50 months.</i>

<i>The customers in the top 20% have a much higher tenure and that is the reason for their high LTV.</i>

---
#### Question 2: What kinds of services did the customers subscribed when they were customers?

#### Service: PhoneService

##### Under 80 percentage (Low LTV customers)

In [16]:
# using SQL
pd.read_sql("select count(1) from churn_all where TotalCharges <= 2840.41 and Churn='Yes' group by PhoneService", conn)

,count(1)
0,161
1,1334


0: No, 1: Yes

In [17]:
# using Pandas value_counts() function
total_charges_under80.PhoneService.value_counts()

Yes    1334
No      161
Name: PhoneService, dtype: int64

In [18]:
# calculating percentage using Pandas value_counts() function
(total_charges_under80.PhoneService.value_counts() / total_charges_under80.shape[0]) * 100

Yes    89.230769
No     10.769231
Name: PhoneService, dtype: float64

~89% of people under 80% of people who left the company did have Phone Service<br>
~11% of people under 80% of people who left the company did not have Phone Service.

##### Above 80 percentage (High LTV customers)

In [19]:
# using Pandas value_counts() function
total_charges_above80.PhoneService.value_counts()

Yes    365
No       9
Name: PhoneService, dtype: int64

In [20]:
# calculating percentage using Pandas value_counts() function
(total_charges_above80.PhoneService.value_counts() / total_charges_above80.shape[0]) * 100

Yes    97.593583
No      2.406417
Name: PhoneService, dtype: float64

~98% of the people above 80% have a Phone Service<br>
~2% of the people above 80% do not have a Phone Service

Insights:<br>
<ol>
    <li>For the <strong>Top 20%</strong> (High LTV customers), only ~2% didn't use Phone Service. </li>
    <li>For the <strong>Bottom 80%</strong> (Low LTV customers), ~11% didn't use Phone Service. </li>
</ol>
<strong>This means having a Phone Service can determine whether a customer is more likely to belong to the High LTV or Low LTV group.</strong>

#### Service: MultipleLines

##### Under 80 percentage (Low LTV customers)

In [21]:
# creating a dataframe where customers have PhoneService. Customers need to have PhoneService in order to have MultipleLines.
total_charges_under80_use_phone = total_charges_under80.query("PhoneService == 'Yes'")

(total_charges_under80_use_phone.MultipleLines.value_counts() / total_charges_under80_use_phone.shape[0]) * 100

No     59.070465
Yes    40.929535
Name: MultipleLines, dtype: float64

~59% of the people below 80% (Low LTV group) did not had Multiple Lines Phone Service.<br>
~41% of the people below 80% (Low LTV group) had Multiple Lines Phone Service.

##### Above 80 percentage (High LTV customers)

In [22]:
# creating a dataframe where customers have PhoneService. Customers need to have PhoneService in order to have MultipleLines.
# above 80 percentile | top 20 percentile 
total_charges_above80_use_phone = total_charges_above80.query("PhoneService == 'Yes'")

(total_charges_above80_use_phone.MultipleLines.value_counts() / total_charges_above80_use_phone.shape[0]) * 100

Yes    83.287671
No     16.712329
Name: MultipleLines, dtype: float64

~83% of the top 20% people (High LTV group) had Multiple Lines Phone Service.<br>
~17% of the top 20% people (High LTV group) did not had Multiple Lines Phone Service.

Insights:<br>
<ol>
    <li>For the <strong>Top 20%</strong> (High LTV customers), only ~17% didn't had Multiple Lines Phone Service. </li>
    <li>For the <strong>Bottom 80%</strong> (Low LTV customers), ~59% didn't had Multiple Lines Phone Service. </li>
</ol>
<strong>This means having a Phone Service with Multiple Lines can get the customers closer to the top 20% (High LTV group).</strong>

#### Service: Internetservice

In [23]:
# calculating percentage using Pandas value_counts() function
(total_charges_under80.InternetService.value_counts() / total_charges_under80.shape[0]) * 100

Fiber optic    64.080268
DSL            28.361204
No              7.558528
Name: InternetService, dtype: float64

In [24]:
# calculating percentage using Pandas value_counts() function
(total_charges_above80.InternetService.value_counts() / total_charges_above80.shape[0]) * 100

Fiber optic    90.641711
DSL             9.358289
Name: InternetService, dtype: float64

All the people in the high 20% LTV used Internet Service. On the other hand only 8% of the people in the low 80% LTV did not have Internet Service. Also, in the top 20%, 90% of them had Fibre Optic in comparison to only 64% in the low LTV. So, as a company if we want to increase the LTV of cutomers, we should recommend using internet and Fibre Optic connection.

#### Service: Other Services

In [25]:
import numpy as np

In [26]:
total_charges_under80_use_internet = total_charges_under80.query('InternetService!="No"')
proportion_internet_sub_service_under80 = np.array([total_charges_under80_use_internet.query('OnlineSecurity=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('TechSupport=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('OnlineBackup=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('DeviceProtection=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('StreamingTV=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0],
total_charges_under80_use_internet.query('StreamingMovies=="Yes"').shape[0]/total_charges_under80_use_internet.shape[0]])

In [27]:
proportion_internet_sub_service_under80

array([0.1316932 , 0.13603473, 0.22720695, 0.23516643, 0.37698987,
       0.3813314 ])

In [28]:
total_charges_above80_use_internet = total_charges_above80.query('InternetService!="No"')
proportion_internet_sub_service_above80 = np.array([total_charges_above80_use_internet.query('OnlineSecurity=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('TechSupport=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('OnlineBackup=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('DeviceProtection=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('StreamingTV=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0],
total_charges_above80_use_internet.query('StreamingMovies=="Yes"').shape[0]/total_charges_above80_use_internet.shape[0]])

In [29]:
proportion_internet_sub_service_above80

array([0.30213904, 0.32620321, 0.55882353, 0.58823529, 0.78342246,
       0.77807487])

In the top 20% of LTV both Streaming Movies and Streaming TV percentages are around 80%, in comparison in the low 80% LTV they are around 40%. Device protection and Online Backup for the top 20% LTV are around 55%, in comparison to only around 22% in the low LTV.

---
#### Question 3: For those who churn what is the proportion of each kind of contracts?

In [30]:
# calculating percentage using Pandas value_counts() function
(total_charges_under80.Contract.value_counts() / total_charges_under80.shape[0]) * 100

Month-to-month    95.919732
One0year           3.678930
Two0year           0.401338
Name: Contract, dtype: float64

In [31]:
# calculating percentage using Pandas value_counts() function
(total_charges_above80.Contract.value_counts() / total_charges_above80.shape[0]) * 100

Month-to-month    59.090909
One0year          29.679144
Two0year          11.229947
Name: Contract, dtype: float64

---
#### Question 4: For those who did not churn what is the proportion of each kind of contracts?

In [32]:
# extracting those customers who churned (churn = no)
paying_df = pd.read_sql("select * from churn_all where Churn = 'No'", conn)

In [33]:
paying_df.TotalCharges.describe()

count    5174.000000
mean     2549.911442
std      2329.954215
min         0.000000
25%       572.900000
50%      1679.525000
75%      4262.850000
max      8672.450000
Name: TotalCharges, dtype: float64

In [34]:
 # finding the 80th percentile of the data in TotalCharges
paying_df.TotalCharges.quantile(0.8)

4890.900000000001

In [35]:
# dividing the data by the 80th percentile of the data from the TotalCharges variable
paying_total_charges_under80 = paying_df.query("TotalCharges <= 4890.9")
paying_total_charges_above80 = paying_df.query("TotalCharges > 4890.9")

In [39]:
# calculating percentage using Pandas value_counts() function
(paying_total_charges_under80.Contract.value_counts() / paying_total_charges_under80.shape[0]) * 100

Month-to-month    49.963759
Two0year          25.465088
One0year          24.571152
Name: Contract, dtype: float64

In [40]:
# calculating percentage using Pandas value_counts() function
(paying_total_charges_above80.Contract.value_counts() / paying_total_charges_above80.shape[0]) * 100

Two0year          57.294686
One0year          28.019324
Month-to-month    14.685990
Name: Contract, dtype: float64

- If we want to stop churning we would like to sell contracts to people for at least 1 year, and much better for 2 years (Conjecture).
- We have 57% of the people that stayed in the company in the top 20% of LTV, and only 11% of the people in the top 20% had a 2 year contract and left the company.
- 95% of the people that left the company and are in the low 80% have a month-to-month contract, in contrast to only 50% of the people in the low 80% that stayed in the company. We can say having a month-to-month contract is one of the reason people are leaving the company (Conjecture)